In [1]:
import pandas as pd
import numpy as np
import requests
import json
import time
from datetime import datetime, timedelta

## Helper functions

In [2]:
import requests
import pandas as pd

def get_books(urls):
    books_ol = []

    for url in urls:
        print(f"Fetching from: {url}")
        try:
            response = requests.get(url)
            response.raise_for_status()
            response_dict = response.json()
        except Exception as e:
            print("Error fetching this URL:", e)
            continue

        for b in response_dict.get("works", []):
            try:
                title = b.get("title", "Unknown Title")
                author = b.get("author_name", ["Unknown Author"])[0]
                year_published = b.get("first_publish_year", None)

                # Fetch ratings
                url_rating = f"https://openlibrary.org/search.json?title={title.replace(' ', '+')}&fields=*"
                response_rating = requests.get(url_rating)
                response_rating_dict = response_rating.json()
                avg_rating = response_rating_dict["docs"][0].get("ratings_average", None)
                rating_count = response_rating_dict["docs"][0].get("ratings_count", None)

                # Fetch subjects / genres
                url_book = f"https://openlibrary.org{b['key']}.json"
                response_book = requests.get(url_book)
                response_book_dict = response_book.json()
                genres = response_book_dict.get("subjects", [])

                for g in genres:
                    books_ol.append({
                        "title": title,
                        "author": author,
                        "avg_rating": avg_rating,
                        "rating_count": rating_count,
                        "year_published": year_published,
                        "genre": g
                    })

            except Exception as e:
                print("there was an error so we continue with the next book:", e)
                continue

    df = pd.DataFrame(books_ol)
    return df


In [3]:
# def get_books(urls):

#     books_ol = []
    
#     for url in urls:
#         print('HI!')
#         for b in response_dict["works"]:
#             try:
#                 author = b['author_name'][0]
#                 title = b['title']
#                 print(title)
#                 year_published = b['first_publish_year']
#                 url_rating = r'https://openlibrary.org/search.json?title='+ title.replace(' ', '+') + '&fields=*'
#                 response_rating = requests.get(url_rating)
#                 response_rating_dict = response_rating.json()
#                 avg_rating = response_rating_dict['docs'][0]['ratings_average']
#                 rating_count = response_rating_dict['docs'][0]['ratings_count']
#                 url_book = r'https://openlibrary.org' + b['key'] + '.json'
#                 response_book = requests.get(url_book)
#                 response_book_dict = response_book.json()
#                 genres = response_book_dict['subjects']
#                 for g in genres:
#                     books_ol.append({'title':title,
#                                      'author':author,
#                                      'avg_rating':avg_rating,
#                                      'rating_count':rating_count,
#                                      'year_published':year_published,
#                                      'genre':g
#                                     })
#             except Exception as e:
#                 print('there was an error so we continue with the next book:', e)
#                 continue
#     df = pd.DataFrame(books_ol)
#     return df     


## Extract data

In [4]:
url_list = ["https://openlibrary.org/trending/yearly.json?offset=0&page=1",
        "https://openlibrary.org/trending/yearly.json?offset=0&page=2",
        "https://openlibrary.org/trending/yearly.json?offset=0&page=3",
        "https://openlibrary.org/trending/yearly.json?offset=0&page=4",
        "https://openlibrary.org/trending/yearly.json?offset=0&page=6",
        "https://openlibrary.org/trending/yearly.json?offset=0&page=7",
        "https://openlibrary.org/trending/yearly.json?offset=0&page=8",
        "https://openlibrary.org/trending/yearly.json?offset=0&page=9",
        "https://openlibrary.org/trending/yearly.json?offset=0&page=10"
       ]

In [5]:
df_books_ol = get_books(url_list) 

Fetching from: https://openlibrary.org/trending/yearly.json?offset=0&page=1
Fetching from: https://openlibrary.org/trending/yearly.json?offset=0&page=2
Fetching from: https://openlibrary.org/trending/yearly.json?offset=0&page=3
Fetching from: https://openlibrary.org/trending/yearly.json?offset=0&page=4
Fetching from: https://openlibrary.org/trending/yearly.json?offset=0&page=6
Fetching from: https://openlibrary.org/trending/yearly.json?offset=0&page=7
there was an error so we continue with the next book: Expecting value: line 1 column 1 (char 0)
Fetching from: https://openlibrary.org/trending/yearly.json?offset=0&page=8
Fetching from: https://openlibrary.org/trending/yearly.json?offset=0&page=9
Fetching from: https://openlibrary.org/trending/yearly.json?offset=0&page=10


In [9]:

print(type(df_books_ol))
print(df_books_ol.head(20))


<class 'pandas.core.frame.DataFrame'>
                   title         author  avg_rating  rating_count  \
0         Kebiasaan Atom    James Clear    3.985531        1244.0   
1         Kebiasaan Atom    James Clear    3.985531        1244.0   
2         Kebiasaan Atom    James Clear    3.985531        1244.0   
3         Kebiasaan Atom    James Clear    3.985531        1244.0   
4         Kebiasaan Atom    James Clear    3.985531        1244.0   
5         Kebiasaan Atom    James Clear    3.985531        1244.0   
6         Kebiasaan Atom    James Clear    3.985531        1244.0   
7         Kebiasaan Atom    James Clear    3.985531        1244.0   
8         Kebiasaan Atom    James Clear    3.985531        1244.0   
9         Kebiasaan Atom    James Clear    3.985531        1244.0   
10        Kebiasaan Atom    James Clear    3.985531        1244.0   
11  The 48 Laws of Power  Robert Greene    4.012915        1084.0   
12  The 48 Laws of Power  Robert Greene    4.012915        1084.0

In [7]:
df_books_ol["title"].nunique()

773

In [11]:
df_books_ol_raw = df_books_ol.to_csv("../data/raw/books_ol_raw.csv",index=False)

In [ ]:
# url = r'https://openlibrary.org/trending/yearly.json?offset=0&page=1'
# response = requests.get(url)
# response_dict = response.json()
# response_dict.keys()

In [ ]:
# type(response_dict["works"])

In [ ]:
# len(response_dict["works"])

In [ ]:
# response_dict["works"][0]

In [ ]:
# author = response_dict['works'][0]['author_name'][0]
# title = response_dict['works'][0]['title']
# year_published = response_dict['works'][0]['first_publish_year']

In [ ]:
# # we create the HTML address based on info available on the website
# url_book = r'https://openlibrary.org' + response_dict['works'][0]['key'] + '.json'
# print(url_book)
# response_book = requests.get(url_book)
# response_book_dict = response_book.json()

In [ ]:
# response_book_dict.keys()

In [ ]:
# response_book_dict['subjects']

In [ ]:


# url_rating = r'https://openlibrary.org/search.json?title='+ response_dict['works'][0]['title'].replace(' ', '+') + '&fields=*'
# url_rating



In [ ]:
# response_rating = requests.get(url_rating)
# response_rating_dict = response_rating.json()

In [ ]:
# response_rating_dict.keys()

In [ ]:
# avg_rating = response_rating_dict['docs'][0]['ratings_average']
# rating_count = response_rating_dict['docs'][0]['ratings_count']